In [1]:
import torch
from torch.utils.data.dataloader import DataLoader

import sys

# Setting path so as we can find files in ../src folder
sys.path.append('../src')

from BRATS2013DatasetPatchMetadata import BRATS2013DatasetPatchMetadata

from tqdm.autonotebook import tqdm

import numpy as np

import json

import skimage

from pprint import pprint

from matplotlib import pyplot as plt

import modin.pandas as pd

import os
os.environ["MODIN_ENGINE"] = "ray"  # Modin will use Dask




In [2]:
DATASET_NAME = "brats_2013"
DATASET_SPLIT = "train"
DATASET_PATH = f"../data/{DATASET_NAME}_obs_path_list_unstacked_resized_cleaned_{DATASET_SPLIT}.txt"

obs_list = []

with open(DATASET_PATH) as file:
  for line in file:
    obs_list.append(line.rstrip())

In [3]:
PATCH_SIZE = 65

In [4]:
dataset = BRATS2013DatasetPatchMetadata(
  obs_list=obs_list, stage=DATASET_SPLIT, patch_size=PATCH_SIZE
)

In [5]:
df_list = []

for entry in tqdm(dataset):
  patch_metadata = entry["patch_metadata"]
  img_path = entry["full_path"]

  for label in patch_metadata:
    for patch_id in patch_metadata[label]:
      
      df_list.append(
        {
          "patch_id": patch_id,
          "label": label,
          "img_path": img_path
        }
      )



0it [00:00, ?it/s]

In [6]:
df = pd.DataFrame(df_list)

2023-01-02 17:09:16,976	INFO worker.py:1538 -- Started a local Ray instance.


In [7]:
df

(raylet) Error processing line 1 of /home/dansolombrino/.local/lib/python3.10/site-packages/modin-autoimport-pandas.pth:
(raylet) 
(raylet)   Traceback (most recent call last):
(raylet)     File "/usr/lib/python3.10/site.py", line 192, in addpackage
(raylet)       exec(line)
(raylet)     File "<string>", line 1, in <module>
(raylet)     File "/home/dansolombrino/.local/lib/python3.10/site-packages/pandas/__init__.py", line 16, in <module>
(raylet)       raise ImportError(
(raylet)   ImportError: Unable to import required dependencies:
(raylet)   pytz: No module named 'pytz'
(raylet) 
(raylet) Remainder of file ignored
(raylet) Error processing line 1 of /home/dansolombrino/.local/lib/python3.10/site-packages/modin-autoimport-pandas.pth:
(raylet) 
(raylet)   Traceback (most recent call last):
(raylet)     File "/usr/lib/python3.10/site.py", line 192, in addpackage
(raylet)       exec(line)
(raylet)     File "<string>", line 1, in <module>
(raylet)     File "/home/dansolombrino/.local/li

,patch_id,label,img_path
0,"[56, 46]",0,../data/BRATS2013_unstacked_resized_slices/000...
1,"[56, 47]",0,../data/BRATS2013_unstacked_resized_slices/000...
2,"[56, 48]",0,../data/BRATS2013_unstacked_resized_slices/000...
3,"[56, 49]",0,../data/BRATS2013_unstacked_resized_slices/000...
4,"[56, 50]",0,../data/BRATS2013_unstacked_resized_slices/000...
...,...,...,...
42443267,"[132, 87]",3,../data/BRATS2013_unstacked_resized_slices/000...
42443268,"[132, 88]",3,../data/BRATS2013_unstacked_resized_slices/000...
42443269,"[132, 89]",3,../data/BRATS2013_unstacked_resized_slices/000...
42443270,"[132, 90]",3,../data/BRATS2013_unstacked_resized_slices/000...


In [8]:
PATCH_DATASET_PATH = f"../data/{DATASET_NAME}_patch_{PATCH_SIZE}_df_{DATASET_SPLIT}.json"

In [9]:
df.to_json(PATCH_DATASET_PATH)

(raylet) [2023-01-02 17:22:16,755 E 42742 42742] (raylet) node_manager.cc:3097: 2 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: 47243dc9d43cd204d2cba968a211d9d30828014ca22a7dd3c6dfc418, IP: 192.168.1.189) over the last time period. To see more information about the Workers killed on this node, use `ray logs raylet.out -ip 192.168.1.189`
(raylet) 
(raylet) Refer to the documentation on how to address the out of memory issue: https://docs.ray.io/en/latest/ray-core/scheduling/ray-oom-prevention.html. Consider provisioning more memory on this node or reducing task parallelism by requesting more CPUs per task. To adjust the kill threshold, set the environment variable `RAY_memory_usage_threshold` when starting Ray. To disable worker killing, set the environment variable `RAY_memory_monitor_refresh_ms` to zero.
